In [11]:
import pandas as pd
import numpy as np

from utils import process_data, compare_models, read_batched_data

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import make_scorer, roc_auc_score

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import pickle

import matplotlib.pyplot as plt

from collections import defaultdict

In [2]:
data_dict = process_data('normal', folds = 5)

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

Beginning data processing ...
Reading in batched data ...


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:12<00:00,  1.47it/s]


Splitting data into train and test ...
Scaling data ...
Completed normal data processing.


## Neural Network Model

In [3]:
def build_NN(num_classes, hidden_layers, neurons_per_layer, dropout_rate, input_dimensions):
    model = Sequential()
    model.add(Dense(neurons_per_layer, input_dim = input_dimensions, activation = 'relu'))

    for i in range(hidden_layers):
        model.add(Dense(neurons_per_layer, activation = 'relu'))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    model.add(Dense(num_classes, activation = 'softmax'))

    model.compile(
        loss = 'sparse_categorical_crossentropy',
        optimizer = 'adam',
        metrics = [tensorflow.keras.metrics.SparseCategoricalAccuracy()]
    )
    
    return model

In [5]:
roc = make_scorer(roc_auc_score, needs_proba = True)
folds = 5

dropout_rates = [0, 0.1, 0.2]
batch_size = [512]
epochs = [10]
hyperparameters = [
    {
        'hidden_layers': [1],
        'neurons_per_layer': [4, 8, 16, 32, 64, 128],
        'dropout_rate': dropout_rates,
        'batch_size': batch_size,
        'epochs': epochs,
    },
    {
        'hidden_layers': [2],
        'neurons_per_layer': [4, 8, 16, 32, 64, 128],
        'dropout_rate': dropout_rates,
        'batch_size': batch_size,
        'epochs': epochs,
    },
    {
        'hidden_layers': [3],
        'neurons_per_layer': [4, 8, 16, 32, 64, 128],
        'dropout_rate': dropout_rates,
        'batch_size': batch_size,
        'epochs': epochs,
    },
]

num_classes = max(data_dict['y_train'])+1
input_dimensions = data_dict['X_train_scaled'].shape[1]

nn_model = KerasClassifier(build_fn = build_NN, num_classes = num_classes, input_dimensions = input_dimensions)
kfolds = StratifiedKFold(n_splits = folds)

grid = GridSearchCV(nn_model, param_grid = hyperparameters, cv = kfolds, scoring = roc, verbose = 10, n_jobs = 1)
model = grid.fit(data_dict['X_train_scaled'], data_dict['y_train'])


<ipython-input-5-89e74c0c5457>:34: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn_model = KerasClassifier(build_fn = build_NN, num_classes = num_classes, input_dimensions = input_dimensions)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5; 1/54] START batch_size=512, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=4
Epoch 1/10
4522/4522 [==============================] - 4s 651us/step - loss: 0.0150 - sparse_categorical_accuracy: 0.9992
Epoch 2/10
4522/4522 [==============================] - 3s 625us/step - loss: 0.0039 - sparse_categorical_accuracy: 0.9992
Epoch 3/10
4522/4522 [==============================] - 3s 614us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9993
Epoch 4/10
4522/4522 [==============================] - 3s 604us/step - loss: 0.0025 - sparse_categorical_accuracy: 0.9995
Epoch 5/10
4522/4522 [==============================] - 3s 607us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 6/10
4522/4522 [==============================] - 3s 608us/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9995
Epoch 7/10
4522/4522 [==============================] - 3s 606us/step - loss: 0.0023 - sparse_categ

4522/4522 [==============================] - 3s 636us/step - loss: 0.0021 - sparse_categorical_accuracy: 0.9995
Epoch 7/10
4522/4522 [==============================] - 3s 636us/step - loss: 0.0021 - sparse_categorical_accuracy: 0.9995
Epoch 8/10
4522/4522 [==============================] - 3s 645us/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9995
Epoch 9/10
4522/4522 [==============================] - 3s 639us/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9996
Epoch 10/10
4522/4522 [==============================] - 3s 636us/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9996
[CV 1/5; 2/54] END batch_size=512, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=8;, score=0.993 total time=  37.2s
[CV 2/5; 2/54] START batch_size=512, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=8
Epoch 1/10
4522/4522 [==============================] - 3s 640us/step - loss: 0.0163 - sparse_categorical_accuracy: 0.9938
Epoch 2/10
4522/4522 [=================

4522/4522 [==============================] - 3s 690us/step - loss: 0.0086 - sparse_categorical_accuracy: 0.9981
Epoch 2/10
4522/4522 [==============================] - 3s 693us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
4522/4522 [==============================] - 3s 715us/step - loss: 0.0022 - sparse_categorical_accuracy: 0.9995
Epoch 4/10
4522/4522 [==============================] - 3s 698us/step - loss: 0.0021 - sparse_categorical_accuracy: 0.9995
Epoch 5/10
4522/4522 [==============================] - 3s 693us/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9996
Epoch 6/10
4522/4522 [==============================] - 3s 723us/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9996
Epoch 7/10
4522/4522 [==============================] - 3s 696us/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9996
Epoch 8/10
4522/4522 [==============================] - 3s 693us/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9996
Epoch 9/10
4522/4522 [=====

4522/4522 [==============================] - 4s 847us/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9996
Epoch 8/10
4522/4522 [==============================] - 3s 767us/step - loss: 0.0018 - sparse_categorical_accuracy: 0.9996
Epoch 9/10
4522/4522 [==============================] - 4s 791us/step - loss: 0.0018 - sparse_categorical_accuracy: 0.9996
Epoch 10/10
4522/4522 [==============================] - 4s 800us/step - loss: 0.0017 - sparse_categorical_accuracy: 0.9996
[CV 2/5; 4/54] END batch_size=512, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=32;, score=0.990 total time=  42.7s
[CV 3/5; 4/54] START batch_size=512, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=32
Epoch 1/10
4522/4522 [==============================] - 4s 774us/step - loss: 0.0062 - sparse_categorical_accuracy: 0.9991
Epoch 2/10
4522/4522 [==============================] - 3s 757us/step - loss: 0.0022 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
4522/4522 [===============

...

4522/4522 [==============================] - 5s 1ms/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9994
Epoch 8/10
4522/4522 [==============================] - 5s 1ms/step - loss: 0.0026 - sparse_categorical_accuracy: 0.9994
Epoch 9/10
4522/4522 [==============================] - 5s 1ms/step - loss: 0.0026 - sparse_categorical_accuracy: 0.9994
Epoch 10/10
4522/4522 [==============================] - 5s 1ms/step - loss: 0.0025 - sparse_categorical_accuracy: 0.9994
[CV 1/5; 50/54] END batch_size=512, dropout_rate=0.2, epochs=10, hidden_layers=3, neurons_per_layer=8;, score=0.991 total time=  59.9s
[CV 2/5; 50/54] START batch_size=512, dropout_rate=0.2, epochs=10, hidden_layers=3, neurons_per_layer=8
Epoch 1/10
4522/4522 [==============================] - 5s 1ms/step - loss: 0.0204 - sparse_categorical_accuracy: 0.9972
Epoch 2/10
4522/4522 [==============================] - 5s 1ms/step - loss: 0.0035 - sparse_categorical_accuracy: 0.9994
Epoch 3/10
4522/4522 [=======================

4522/4522 [==============================] - 7s 2ms/step - loss: 0.0031 - sparse_categorical_accuracy: 0.9992
Epoch 3/10
4522/4522 [==============================] - 7s 2ms/step - loss: 0.0029 - sparse_categorical_accuracy: 0.9992
Epoch 4/10
4522/4522 [==============================] - 7s 2ms/step - loss: 0.0028 - sparse_categorical_accuracy: 0.9995
Epoch 5/10
4522/4522 [==============================] - 7s 2ms/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9995
Epoch 6/10
4522/4522 [==============================] - 7s 2ms/step - loss: 0.0026 - sparse_categorical_accuracy: 0.9995
Epoch 7/10
4522/4522 [==============================] - 7s 2ms/step - loss: 0.0026 - sparse_categorical_accuracy: 0.9995
Epoch 8/10
4522/4522 [==============================] - 7s 2ms/step - loss: 0.0025 - sparse_categorical_accuracy: 0.9995
Epoch 9/10
4522/4522 [==============================] - 7s 2ms/step - loss: 0.0025 - sparse_categorical_accuracy: 0.9995
Epoch 10/10
4522/4522 [====================

4522/4522 [==============================] - 6s 1ms/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9996
Epoch 10/10
4522/4522 [==============================] - 6s 1ms/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9996
[CV 2/5; 52/54] END batch_size=512, dropout_rate=0.2, epochs=10, hidden_layers=3, neurons_per_layer=32;, score=0.991 total time= 1.2min
[CV 3/5; 52/54] START batch_size=512, dropout_rate=0.2, epochs=10, hidden_layers=3, neurons_per_layer=32
Epoch 1/10
4522/4522 [==============================] - 7s 1ms/step - loss: 0.0049 - sparse_categorical_accuracy: 0.9994TA: 8s - loss: 0.0672 - sparse_categorical - ETA: 5s - l
Epoch 2/10
4522/4522 [==============================] - 7s 1ms/step - loss: 0.0025 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
4522/4522 [==============================] - 7s 1ms/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9995
Epoch 4/10
4522/4522 [==============================] - 7s 1ms/step - loss: 0.0022 - sparse_categorical_accu

4522/4522 [==============================] - 9s 2ms/step - loss: 0.0021 - sparse_categorical_accuracy: 0.9996
Epoch 5/10
4522/4522 [==============================] - 8s 2ms/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9996
Epoch 6/10
4522/4522 [==============================] - 8s 2ms/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9996
Epoch 7/10
4522/4522 [==============================] - 8s 2ms/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9996
Epoch 8/10
4522/4522 [==============================] - 8s 2ms/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9996
Epoch 9/10
4522/4522 [==============================] - 8s 2ms/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9996
Epoch 10/10
4522/4522 [==============================] - 8s 2ms/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9996
[CV 3/5; 53/54] END batch_size=512, dropout_rate=0.2, epochs=10, hidden_layers=3, neurons_per_layer=64;, score=0.989 total time= 1.5min
[CV 4/5; 53/54] START batch

Epoch 1/10
4522/4522 [==============================] - 13s 3ms/step - loss: 0.0038 - sparse_categorical_accuracy: 0.9992
Epoch 2/10
4522/4522 [==============================] - 12s 3ms/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
4522/4522 [==============================] - 12s 3ms/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9995
Epoch 4/10
4522/4522 [==============================] - 13s 3ms/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9995
Epoch 5/10
4522/4522 [==============================] - 13s 3ms/step - loss: 0.0022 - sparse_categorical_accuracy: 0.9996
Epoch 6/10
4522/4522 [==============================] - 13s 3ms/step - loss: 0.0021 - sparse_categorical_accuracy: 0.9996
Epoch 7/10
4522/4522 [==============================] - 13s 3ms/step - loss: 0.0021 - sparse_categorical_accuracy: 0.9996
Epoch 8/10
4522/4522 [==============================] - 14s 3ms/step - loss: 0.0021 - sparse_categorical_accuracy: 0.9996
Epoch 9/10
4522/4522 [==

In [6]:
nn_results = {
    'cv_results_': grid.cv_results_,
    'best_score_': grid.best_score_,
    'best_params_': grid.best_params_,
    'best_index_': grid.best_index_,
    'scorer_': grid.scorer_,
    'n_splits_': grid.n_splits_,
}

In [7]:
cv_results = defaultdict(list)
for i, mean_test_score in enumerate(grid.cv_results_['mean_test_score']):
    cv_results['dropout_rate'].append(grid.cv_results_['params'][i]['dropout_rate'])
    cv_results['hidden_layers'].append(grid.cv_results_['params'][i]['hidden_layers'])
    cv_results['neurons_per_layer'].append(grid.cv_results_['params'][i]['neurons_per_layer'])
    cv_results['mean_test_score'].append(mean_test_score)
    
nn_cv_results_df = pd.DataFrame(cv_results)
display(nn_cv_results_df)

,dropout_rate,hidden_layers,neurons_per_layer,mean_test_score
0,0.0,1,4,0.992298
1,0.0,1,8,0.991548
2,0.0,1,16,0.991752
3,0.0,1,32,0.991767
4,0.0,1,64,0.990531
5,0.0,1,128,0.990174
6,0.1,1,4,0.990343
7,0.1,1,8,0.992196
8,0.1,1,16,0.991919
9,0.1,1,32,0.992404


In [28]:
print(f'Best ROC AUC Score: {grid.best_score_:.6f}')

Best ROC AUC Score: 0.992594


In [25]:
print('Best NN Parameters:')
print(grid.best_params_)

Best NN Parameters:
{'batch_size': 512, 'dropout_rate': 0.1, 'epochs': 10, 'hidden_layers': 2, 'neurons_per_layer': 16}


In [9]:
with open('neural-network-gridsearchcv-results-v2.pkl', 'wb') as handle:
    pickle.dump(nn_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
# final_model = build_NN(num_classes, model.best_params_['hidden_layers'], model.best_params_['neurons_per_layer'], model.best_params_['dropout_rate'], input_dimensions)
# final_model.fit(x_train, y_train, batch_size = model.best_params_['batch_size'], epochs = model.best_params_['epochs'])

In [10]:
with open('neural-network-model-v2.json', 'w') as json_file:
    json_file.write(grid.best_estimator_.model.to_json())
grid.best_estimator_.model.save_weights('neural-network-model-v2.h5')

#### Testing Performance

In [20]:
clfs = [grid]
clf_names = ['neural_network']

compare_models(
    clfs,
    clf_names,
    data_dict['X_train_scaled'],
    data_dict['y_train'],
    data_dict['X_test_scaled'],
    data_dict['y_test'],
)

,neural_network
accuracy_test,0.999588
accuracy_train,0.999604
precision_test,0.911765
precision_train,0.928811
recall_test,0.562706
recall_train,0.571016
roc_auc_test,0.989228
roc_auc_train,0.994140
